In [2]:
using JuMP, Cbc

In [98]:
N = [2 3 1]  #tasks
W = hcat(ones(1), 3*ones(1), 2*ones(1)) # 1 worker who completes task 1 in 1 time, tasks 2 in three time, etc.
T = convert(Int64,maximum(W)*sum(N)) #upper bound on project completion

m = Model(solver=CbcSolver())

n = maximum(N) #max number of tasks for a category
v = size(N,2) # number of types of tasks
w = size(W,1) #number of workers

@variable(m, x[1:n, 1:v, 1:w, 1:T], Bin)

@variable(m, last)

# due to the following two constraints, this just selects the time
# when x[j, i, t] = 1, since x[j, :, :] has only one nonzero elemnt
#@expression(m, start_t[j=1:n], sum(x[j,:,:,:]))
@expression(m, end_t[j=1:n, k=1:v, i=1:w], sum((x[j,k,i,t])*(t+W[i,k]) for t=1:T))

# at most one job per timestep per person
@constraint(m, [t=1:T, i=1:w], sum(x[:, :, i, t]) <= 1)

# each task is only worked on once and all tasks are completed
#@constraint(m, [j=1:n, i=1:v], sum(x[j, i, :, :]) <= 1)
@constraint(m, [j=1:n, i=1:v], sum(x[j, i, :, :]) <= 1)
@constraint(m, [i=1:v], sum(x[:,i,:,:]) == N[i])

# min { max_j end_t[j] }
@constraint(m, [j=1:n, k=1:v, i=1:w], last >= end_t[j,k,w])

@objective(m, Min, last)

solve(m)
getobjectivevalue(m)

7.0

In [100]:
#m
#getvalue(x[:,1,1,:])
#(getvalue(start_t))
#x[1,1,:,:]
#getvalue(end_t)



In [97]:
getvalue(end_t)
#getvalue(x[:,:,1,5])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 25.0  57.0  36.0
 18.0  56.0  41.0
 22.0  55.0  36.0